# Rowing Robot 

In [3]:
import sympy as sm
import sympy.physics.mechanics as me
me.init_vprinting()

## Variables

In [4]:
# lim lengths
lower_leg_length, upper_leg_length, torso_length = sm.symbols('l_L, l_U, l_T')

# lim masses
lower_leg_mass, upper_leg_mass, torso_mass = sm.symbols('m_L, m_U, m_T')

# joint angles
theta1, theta2, theta3 = me.dynamicsymbols('theta1, theta2, theta3')

# joint torques
ankle_torque, knee_torque, hip_torque = me.dynamicsymbols('T_a, T_k, T_h')

# Center of mass (com)
lower_leg_com_length, upper_leg_com_length, torso_com_length = sm.symbols('d_L, d_U, d_T')
torso_com_leght = torso_length / 2

# joint angle velocities
omega1, omega2, omega3 = me.dynamicsymbols('omega1, omega2, omega3')

# physics symbos
g = sm.symbols('g')

## Reference Frames

In [5]:
inertial_frame = me.ReferenceFrame('N')  # boat
lower_leg_frame = me.ReferenceFrame('L')  # lower leg
upper_leg_frame = me.ReferenceFrame('U')  # upper leg 
torso_frame = me.ReferenceFrame('T')  # torso

In [6]:
lower_leg_frame.orient(inertial_frame, 'Axis', (theta1, inertial_frame.z))
upper_leg_frame.orient(lower_leg_frame, 'Axis', (theta2, lower_leg_frame.z))
torso_frame.orient(upper_leg_frame, 'Axis', (theta3, upper_leg_frame.z))

In [7]:
sm.simplify(torso_frame.dcm(inertial_frame))

⎡cos(θ₁ + θ₂ + θ₃)   sin(θ₁ + θ₂ + θ₃)  0⎤
⎢                                        ⎥
⎢-sin(θ₁ + θ₂ + θ₃)  cos(θ₁ + θ₂ + θ₃)  0⎥
⎢                                        ⎥
⎣        0                   0          1⎦

# Configuration Constrains

## Joint points

In [8]:
ankle = me.Point('A')
knee = me.Point('K')
hip = me.Point('H')
head = me.Point('N')  # N for Noggin

In [9]:
knee.set_pos(ankle, lower_leg_length * lower_leg_frame.y)
hip.set_pos(knee, upper_leg_length * upper_leg_frame.y)
head.set_pos(hip, torso_length * torso_frame.y)

## Center of mass points

In [10]:
lower_leg_mass_center = me.Point('L_o')
lower_leg_mass_center.set_pos(ankle, lower_leg_com_length * lower_leg_frame.y)

upper_leg_mass_center = me.Point('U_o')
upper_leg_mass_center.set_pos(knee, upper_leg_com_length * upper_leg_frame.y)

torso_mass_center = me.Point('T_o')
torso_mass_center.set_pos(hip, torso_com_length * torso_frame.y)

## Configuration Constrains with joint system

loop: link feet, knee, hip and head. by means of lower leg, upper leg and body.


In [11]:
# loop_angle2hip = P3.pos_from(P1)
#loop_angle2hip

# Kinematics Differential Equations

### Generalized Velocities

In [12]:
kinematical_differential_equations = [omega1 - theta1.diff(),
                                      omega2 - theta2.diff(),
                                      omega3 - theta3.diff()]

### Angular Velocities

In [13]:
lower_leg_frame.set_ang_vel(inertial_frame, omega1 * inertial_frame.z)
lower_leg_frame.ang_vel_in(inertial_frame)

upper_leg_frame.set_ang_vel(lower_leg_frame, omega2 * inertial_frame.z)
upper_leg_frame.ang_vel_in(inertial_frame)

torso_frame.set_ang_vel(upper_leg_frame, omega3 * inertial_frame.z)
torso_frame.ang_vel_in(inertial_frame)

### Linear Velocities

In [14]:
ankle.set_vel(inertial_frame, 0)
lower_leg_mass_center.v2pt_theory(ankle, inertial_frame, lower_leg_frame)
knee.v2pt_theory(ankle, inertial_frame, lower_leg_frame)
upper_leg_mass_center.v2pt_theory(knee, inertial_frame, upper_leg_frame)
hip.v2pt_theory(knee, inertial_frame, upper_leg_frame)
torso_mass_center.v2pt_theory(hip, inertial_frame, torso_frame)

# Inertia

## Inertia

In [15]:
lower_leg_inertia, upper_leg_inertia, torso_inertia = sm.symbols('I_Lz, I_Uz, I_Tz')

In [16]:
lower_leg_inertia_dyadic = me.inertia(lower_leg_frame, 0, 0, lower_leg_inertia)
lower_leg_central_inertia = (lower_leg_inertia_dyadic, lower_leg_mass_center)

upper_leg_inertia_dyadic = me.inertia(upper_leg_frame, 0, 0, upper_leg_inertia)
upper_leg_central_inertia = (upper_leg_inertia_dyadic, upper_leg_mass_center)

torso_inertia_dyadic = me.inertia(torso_frame, 0, 0, torso_inertia)
torso_central_inertia = (torso_inertia_dyadic, torso_mass_center)

## Rigig Bodies

In [17]:
lower_leg = me.RigidBody('Lower Leg', lower_leg_mass_center, lower_leg_frame,
                      lower_leg_mass, lower_leg_central_inertia)
upper_leg = me.RigidBody('Upper Leg', upper_leg_mass_center, upper_leg_frame,
                      upper_leg_mass, upper_leg_central_inertia)

torso = me.RigidBody('Torso', torso_mass_center, torso_frame,
                  torso_mass, torso_central_inertia)

# Kinetics

In [18]:
lower_leg_grav_force = (lower_leg_mass_center, -lower_leg_mass * g * inertial_frame.y)
upper_leg_grav_force = (upper_leg_mass_center, -upper_leg_mass * g * inertial_frame.y)
torso_grav_force = (torso_mass_center, -torso_mass * g * inertial_frame.y)

## Joint Torques

In [19]:
lower_leg_torque = (lower_leg_frame, ankle_torque * inertial_frame.z - knee_torque * inertial_frame.z)
upper_leg_torque = (upper_leg_frame, knee_torque * inertial_frame.z - hip_torque * inertial_frame.z)
torso_torque = (torso_frame, hip_torque * inertial_frame.z)

# Motion Constrains (aka 'seat configuration')

Have the hips fixed to the bench, aka only movement in x-direction.

# Description of system

In [20]:
kwargs = dict(
    q_ind=[theta1, theta2, theta3], # independent generalized coordinates
    u_ind=[omega1, omega2, omega3], # independent generalized speed
    kd_eqs=kinematical_differential_equations, # q' = u for all coordinates
)

kane = me.KanesMethod(inertial_frame, # inertial reference frame
                     **kwargs)

In [22]:
#kane.form_eoms()

In [23]:
bodies = [lower_leg, upper_leg, torso]

In [24]:
loads = [lower_leg_grav_force,
         upper_leg_grav_force,
         torso_grav_force, 
         lower_leg_torque,
         upper_leg_torque,
         torso_torque]

In [25]:
fr, frstar = kane.kanes_equations(bodies, loads=loads)

In [26]:
# sm.trigsimp(fr + frstar)
fr

⎡d_L⋅g⋅m_L⋅sin(θ₁) + d_T⋅g⋅m_T⋅((-sin(θ₂)⋅sin(θ₃) + cos(θ₂)⋅cos(θ₃))⋅sin(θ₁) +
⎢                                                                             
⎢                              d_T⋅g⋅m_T⋅((-sin(θ₂)⋅sin(θ₃) + cos(θ₂)⋅cos(θ₃))
⎢                                                                             
⎣                                                                             

 (sin(θ₂)⋅cos(θ₃) + sin(θ₃)⋅cos(θ₂))⋅cos(θ₁)) + d_U⋅g⋅m_U⋅(sin(θ₁)⋅cos(θ₂) + s
                                                                              
⋅sin(θ₁) + (sin(θ₂)⋅cos(θ₃) + sin(θ₃)⋅cos(θ₂))⋅cos(θ₁)) + d_U⋅g⋅m_U⋅(sin(θ₁)⋅c
                                                                              
 d_T⋅g⋅m_T⋅((-sin(θ₂)⋅sin(θ₃) + cos(θ₂)⋅cos(θ₃))⋅sin(θ₁) + (sin(θ₂)⋅cos(θ₃) + 

in(θ₂)⋅cos(θ₁)) + g⋅l_L⋅m_T⋅sin(θ₁) + g⋅l_L⋅m_U⋅sin(θ₁) + g⋅l_U⋅m_T⋅(sin(θ₁)⋅c
                                                                              
os(θ₂) + sin(θ₂)⋅cos(θ₁)) + g⋅l_U⋅m_T⋅(sin(θ₁)⋅cos

# Solving and plotting

In [ ]:
import numpy as np
from scipy.optimize import fsolve
from pydy.system import System

In [ ]:
duration = 5.0
fps = 30.0
times = np.linspace(0.0, duration, num=int(fps*duration))

In [ ]:
sys = System(kane, times=times)

In [ ]:
def step_pulse(x, t):
    if t < 2.0:
        T = 5.0
    else:
        T = 0.0
    return np.array([0.0])

In [ ]:
constants = {lower_leg_length: 0.611,
             lower_leg_com_length: 0.387,
             lower_leg_mass: 6.769,
             lower_leg_inertia: 0.101,
             upper_leg_length: 0.424,
             upper_leg_com_length: 0.193,
             upper_leg_mass: 17.01,
             upper_leg_inertia: 0.282,
             torso_com_length: 0.305,
             torso_mass: 32.44,
             torso_inertia: 1.485,
             g: 9.81}

sys.constants = constants

In [ ]:
# q1_0 = np.deg2rad(90.0)
# q2_0 = np.deg2rad(120.0)
# q3_0 = np.deg2rad(90.0)
# q1_0 = np.deg2rad(0.0)
# q2_0 = np.deg2rad(0.0)
# q3_0 = np.deg2rad(0.0)

In [ ]:
sys.initial_conditions = {theta1: np.deg2rad(0.0),
                          theta2: np.deg2rad(-120.0),
                          theta3: np.deg2rad(120.0),
                          omega1: .0,
                          omega2: .0,
                          omega3: .0,
                         }

In [ ]:
sys.specifieds = {ankle_torque: step_pulse, knee_torque: step_pulse, hip_torque: step_pulse}

In [ ]:
x = sys.integrate()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, axes = plt.subplots(3, 2, sharex=True)
fig.set_size_inches(10, 10)

for i, (xi, ax, s) in enumerate(zip(x.T, axes.T.flatten(), sys.states)):
    ax.plot(sys.times, np.rad2deg(xi))
    title = sm.latex(s, mode='inline')
    ax.set_title(title)
    if 'q' in title:
        ax.set_ylabel('Angle [deg]')
    else:
        ax.set_ylabel('Angular Rate [deg/s]')

axes[1, 0].set_xlabel('Time [s]')
axes[1, 1].set_xlabel('Time [s]')

plt.tight_layout()

# Animation

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [ ]:
knee.pos_from(ankle).express(inertial_frame).simplify()

In [ ]:
hip.pos_from(knee).express(inertial_frame).simplify()

In [ ]:
head.pos_from(hip).express(inertial_frame).simplify()


In [ ]:
q1_vals = x[:, 0]
q2_vals = x[:, 1]
q3_vals = x[:, 2]

p2_xy = np.array([-constants[lower_leg_length]*np.sin(q1_vals),
                   constants[lower_leg_length]*np.cos(q1_vals)])

p3_xy = p2_xy + np.array([-constants[upper_leg_length]*np.sin(q1_vals + q2_vals),
                          constants[upper_leg_length]*np.cos(q1_vals + q2_vals)])

p4_xy = p3_xy + np.array([-2 * constants[torso_com_length]*np.sin(q1_vals + q2_vals + q3_vals),
                           2 * constants[torso_com_length]*np.cos(q1_vals + q2_vals + q3_vals)])



In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches((6, 5))
line, = ax.plot([0.0, p2_xy[0, 0], p3_xy[0, 0], p4_xy[0, 0]],
                [0.0, p2_xy[1, 0], p3_xy[1, 0], p4_xy[1, 0]])
title = 'Time = {:0.1f} seconds'
ax.set_title(title.format(0.0))
ax.set_ylim((-2.0, 2.0))
ax.set_xlim((-2.0, 2.0))
ax.set_aspect('equal')

In [ ]:
def update(i):
    xdata = [0.0, p2_xy[0, i], p3_xy[0, i], p4_xy[0, i]]
    ydata = [0.0, p2_xy[1, i], p3_xy[1, i], p4_xy[1, i]]
    line.set_data(xdata, ydata)
    ax.set_title(title.format(sys.times[i]))
    return line,

ani = FuncAnimation(fig, update, save_count=len(sys.times))

In [ ]:
HTML(ani.to_jshtml(fps=fps))